In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "clientid",
"fs.azure.account.oauth2.client.secret": 'secretkey',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenantid/oauth2/token"}

dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@tokyoolympicdataarya.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)


In [ ]:

%fs
ls "/mnt/tokyoolymic"


In [ ]:
spark

In [ ]:
athletes=spark.read.format("csv").option("header","True").option("inferSchema","True").load("/mnt/tokyoolymic/raw-data/athletes.csv")
coaches=spark.read.format("csv").option("header","True").option("inferSchema","True").load("/mnt/tokyoolymic/raw-data/coaches.csv")
entriesgender=spark.read.format("csv").option("header","True").option("inferSchema","True").load("/mnt/tokyoolymic/raw-data/entriesgender.csv")
medals=spark.read.format("csv").option("header","True").option("inferSchema","True").load("/mnt/tokyoolymic/raw-data/medals.csv")
teams=spark.read.format("csv").option("header","True").option("inferSchema","True").load("/mnt/tokyoolymic/raw-data/teams.csv")

In [ ]:
athletes.show()

In [ ]:
athletes.printSchema()

In [ ]:
coaches.show()

In [ ]:
coaches.printSchema()

In [ ]:
entriesgender.show()

In [ ]:
entriesgender.printSchema()

In [ ]:
entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))

In [ ]:
entriesgender.printSchema()

In [ ]:
medals.show()

In [ ]:
medals.printSchema()

In [ ]:
teams.show()

In [ ]:
teams.printSchema()

In [ ]:
# Find the countries with the highest number of gold medals
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team/NOC","Gold").show()

In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

In [ ]:
# to avoid saving metadata, convert into pandas df first

# load transformed data to the target location
athletes.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/athletes.csv")
coaches.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/coaches.csv")
entriesgender.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/entriesgender.csv")
medals.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/medals.csv")
teams.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/teams.csv")